In [1]:
import pandas as pd 
from openai import OpenAI
import json

In [2]:
# 設定OpenAI API
client = OpenAI(
    api_key = 'xxxxxxxx'
)


In [3]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "..."},
    {"role": "user", "content": "請問 Python 是一個怎樣的程式語言？"}
  ]
)

print(completion.choices[0].message.content)

Python 是一種高級、直譯式的程式語言，具有簡潔易讀的語法，適合初學者和專業開發人員使用。以下是 Python 的一些特點：

1. **簡單易讀**：Python 的語法設計強調可讀性，這使得程式碼更容易理解和維護。

2. **多範式支援**：Python 支援多種程式設計範式，包括物件導向、函數式和命令式編程。

3. **豐富的標準庫**：Python 附帶了一個功能強大的標準庫，涵蓋了多種功能，如文件操作、數據處理、網路通訊、正則表達式等。

4. **跨平台**：Python 可以在多種作業系統上運行，包括 Windows、macOS 和 Linux。

5. **社群活躍**：Python 擁有龐大的使用者社群，提供豐富的資源和第三方庫，例如 NumPy、Pandas、Django 和 Flask 等。

6. **廣泛應用**：Python 被廣泛應用於網頁開發、數據分析、人工智慧、機器學習、自動化腳本、科學計算等領域。

7. **動態類型**：Python 是一種動態類型的語言，這意味著變數不需要明確聲明類型，類型是在執行時決定的。

總的來說，Python 是一種功能強大且靈活的編程語言，受到廣泛的喜愛和應用。


In [ ]:
# 函數：讀取CSV文件
def load_csv_to_dataframe(file_path='tables_and_columns.csv'):
    df = pd.read_csv(file_path)
    return df

# 函數：根據用戶查詢進行數據檢索
def query_data(df, intent, parameters):
    if any(keyword in intent for keyword in ["查詢","顯示"]):
        if any(keyword in parameters.get("對象") for keyword in ["表單", "tables"]):
            tables = df['table_name'].unique()
            return tables
        elif any(keyword in parameters.get("操作") for keyword in ["欄位", "columns"]) and "對象" in parameters:
            table_name = parameters["對象"]
            columns = df[df['table_name'] == table_name]['column_name'].unique()
            return columns
    return "查詢無結果"



# 函數：從用戶輸入中解析意圖和參數
def parse_intent_with_gpt(user_input):
    prompt = f"""
    你是一個有用的助手，能夠解析用戶輸入並提取意圖和參數。
    請將以下用戶輸入解析成包含意圖和參數的JSON格式：
    用戶輸入: "{user_input}"
    返回格式: {{"intent": "<意圖>", "parameters": {{"對象": "<表單名>", "操作": "<欄位>"}}}}
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "你是一個有用的助手，能夠解析用戶輸入並提取意圖和參數。"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0.5
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

# 函數：處理用戶輸入
def process_user_input(user_input):
    # 在這裡調用 load_csv_to_dataframe 函數
    df = load_csv_to_dataframe()

    # 調用 GPT 模型解析意圖和參數
    gpt_response = parse_intent_with_gpt(user_input)
    print(f"GPT解析結果: {gpt_response}")

    # 假設 GPT 返回的結果是 JSON 格式，解析意圖和參數
    intent, parameters = parse_gpt_response(gpt_response)
    print(f"解析出的意圖: {intent}")
    print(f"解析出的參數: {parameters}")

    # 根據意圖和參數進行查詢
    result = query_data(df, intent, parameters)
    return result

# 模擬解析 GPT 返回的 JSON 內容
def parse_gpt_response(gpt_response):
    parsed_response = json.loads(gpt_response)
    intent = parsed_response.get("intent")
    parameters = parsed_response.get("parameters", {})
    return intent, parameters

# 主函數
def main():
    # 獲取用戶查詢
    user_query = input("請輸入查詢內容: ")  # 假設這是使用者的輸入

    # 處理用戶輸入
    result = process_user_input(user_query)
    print(f"查詢結果: {result}")

if __name__ == "__main__":
    main()


{
    "intent": "查詢",
    "parameters": {
        "對象": "employee",
        "操作": "欄位"
    }
}
GPT解析結果: {
    "intent": "查詢",
    "parameters": {
        "對象": "employee",
        "操作": "欄位"
    }
}
解析出的意圖: 查詢
解析出的參數: {'對象': 'employee', '操作': '欄位'}
查詢結果: ['ID' 'DEPT_ID' 'SALARY']
